In [ ]:
import pandas as pd 
import math
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.regularizers import l2
import tensorflow as tf

In [ ]:
df = pd.read_csv("../input/bravespositioning/braves_23_sim.csv")
pd.set_option('display.max_columns', None)
df

In [ ]:
# GROUND BALL MODEL
gb_df = df[(df['is_gb'] == 1)]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Embedding, Flatten, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Simplified feature set with added 'hit_direction' and positional features
features_simplified = [
    'responsible_fielder',
    'launch_speed',
    'responsible_fielder_angle_diff', 'responsible_fielder_depth',
]
target = 'is_out'
# Separate features and target
X_simplified = gb_df[features_simplified].copy()
y = gb_df[target]

# Encode the categorical feature 'responsible_fielder'
label_encoder = LabelEncoder()
X_simplified['responsible_fielder'] = label_encoder.fit_transform(X_simplified['responsible_fielder'])

# Define numeric features in the simplified set
numeric_features_simplified = [
    'launch_speed', 
    'responsible_fielder_angle_diff', 'responsible_fielder_depth',
]

# Preprocessing pipeline for the simplified numeric data
numeric_transformer_simplified = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor_simplified = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_simplified, numeric_features_simplified)
    ])

# Split the data with the simplified features
X_train_simplified, X_test_simplified, y_train, y_test = train_test_split(X_simplified, y, test_size=0.2, random_state=42)

# Apply preprocessing to numeric features
X_train_processed_simplified = preprocessor_simplified.fit_transform(X_train_simplified)
X_test_processed_simplified = preprocessor_simplified.transform(X_test_simplified)

from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Embedding, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define input layers for numeric and categorical features
numeric_input = Input(shape=(X_train_processed_simplified.shape[1],), name='numeric_input')
embedded_fielder = Embedding(input_dim=len(label_encoder.classes_), output_dim=5, input_length=1)(X_train_simplified['responsible_fielder'])
embedded_fielder = Flatten()(embedded_fielder)

# Concatenate numeric and embedded features
concatenated = concatenate([numeric_input, embedded_fielder])

# Build the model
model = Sequential([
    Dense(256, activation='relu', kernel_regularizer=l1_l2(l1=0.001, l2=0.0005)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=0.001, l2=0.0005)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=0.001, l2=0.0005)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)

# Train the model
history = model.fit(X_train_processed_simplified, y_train, epochs=110, batch_size=44, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_processed_simplified, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Plotting the training history
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()



In [ ]:
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt

# Predict probabilities for the test set
y_prob = model.predict(X_test_processed_simplified)

# Since the output is a probability, ensure it's 1-dimensional
y_prob = y_prob.flatten()

# Calculate the calibration curve
fraction_of_positives, mean_predicted_value = calibration_curve(y_test, y_prob, n_bins=10)

# Plotting the calibration plot
plt.figure(figsize=(10, 7))
plt.plot(mean_predicted_value, fraction_of_positives, "s-", label="Model Calibration")
plt.plot([0, 1], [0, 1], "k--", label="Perfect Calibration")
plt.xlabel("Mean Predicted Probability")
plt.ylabel("Fraction of Positives")
plt.title("Calibration Plot")
plt.legend()
plt.show()



In [ ]:
# This got really confusing with the model up above, so I decided to just redo all the preprocessing and then use our model for predictions 
X_all_num = preprocessor.fit_transform(X_simplified[numeric_features])
predictions = model.predict([X_all_num, X_simplified['responsible_fielder']]).flatten()
gb_df['predicted_out_probability'] = predictions
gb_df

In [ ]:
# INFIELD PLOT 
lauch_speed = 85
#YOU CHOOSE FIELDER X and Y and responsible rielder
fielder = 8 #CFer
x,y = np.meshgrid(np.linspace(fielder_x-120,fielder_x + 120,1000), np.linspace(fielder_y - 120,fielder_y + 120,1000))
x = x.flatten()
y = y.flatten()
hit_direction_angle = np.arctan2(x,y)
new_df = pd.DataFrame({'responsible_fielder': np.ones(x.shape[0]) * fielder,
                        'hit_direction': hit_direction_angle,
                      'landing_x': x, 'landing_y': y,
                       'responsible_fielder_x': np.ones(x.shape[0]) * fielder_x,
                      'responsible_fielder_y': np.ones(x.shape[0]) * fielder_y})
new_df.responsible_fielder = new_df.responsible_fielder.astype(int)
#TODO: run new_df through model and get out prob columns
X = new_df[features].copy()  # Avoid modifying the original DataFrame

# Encode the categorical feature `responsible_fielder`
#label_encoder = LabelEncoder()
X['responsible_fielder'] = label_encoder.transform(X['responsible_fielder'])

# Define numeric features
# numeric_features = [
#     'hit_direction', 'hangtime', 'landing_x', 'landing_y',
#     'responsible_fielder_x', 'responsible_fielder_y'
# ]
# Create a preprocessing pipeline for numeric data
# numeric_transformer = Pipeline(steps=[
#     ('scaler', StandardScaler())
# ])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features)
#     ])

X_all_num = preprocessor.transform(X[numeric_features])
new_df['outprob'] = model.predict([X_all_num, new_df['responsible_fielder']]).flatten()
outprob = new_df.outprob.values
outprob = outprob.reshape((1000,1000))
x = x.reshape((1000,1000))
y = y.reshape((1000,1000))
plt.pcolormesh(x,y,outprob, cmap = 'coolwarm')
plt.colorbar(label = 'out probability')
plt.xlabel('landing x')
plt.ylabel('landing y')
plt.plot([fielder_x], [fielder_y], 'ko')
plt.show()
